In [2]:
import pandas as pd
import numpy as np
from math import trunc
import matplotlib.pyplot as plt
%matplotlib qt
import argparse, sys, os
import pygeodesy
from scipy.signal import butter, filtfilt, resample
from scipy.spatial.distance import cdist
from scipy.interpolate import make_interp_spline, make_smoothing_spline, BSpline
from scipy.ndimage import uniform_filter1d, spline_filter1d
from scipy.ndimage import gaussian_filter1d, uniform_filter1d


In [3]:
# current_path = os.path.dirname(__file__)

#put your relative path to the csv files
# raptor_msgs = pd.read_csv('../data/pp_run4_2_raptor.csv')
# imu_msgs = pd.read_csv('../data/pp_run4_2_imu.csv')
raptor_msgs = pd.read_csv('../data/lvms_brake_test_raptor_msgs.csv')
imu_msgs = pd.read_csv('../data/lvms_brake_test_imu_inputs.csv')
print(list(raptor_msgs.columns))
print(list(imu_msgs.columns))

def normalize_data(input_times, data):
    input_times = input_times[~np.isnan(data)]
    data = data[~np.isnan(data)]
    return np.column_stack([input_times, data])

raptor_times = np.array(raptor_msgs['__time'])
imu_times = np.array(imu_msgs['__time'])

raptor_times = np.round(raptor_times - imu_times[0], 3)
imu_times = np.round(imu_times - imu_times[0], 3)

KPH2MPH = 0.621371
rear_right_wheel_speed = np.array(raptor_msgs['/raptor_dbw_interface/wheel_speed_report/rear_right'])
rear_left_wheel_speed = np.array(raptor_msgs['/raptor_dbw_interface/wheel_speed_report/rear_left'])

rear_avg_wheel_speed = (rear_right_wheel_speed + rear_left_wheel_speed) / 2
# desired_vel = np.array(imu_msgs['/long_control/set_desired_velocity/data'])
rear_avg_wheel_speed = normalize_data(raptor_times, rear_avg_wheel_speed * KPH2MPH)
# desired_vel = normalize_data(imu_times, desired_vel)

filtered_rear_avg_wheel_speed = np.array(rear_avg_wheel_speed)
filtered_rear_avg_wheel_speed[:, 1] = gaussian_filter1d(rear_avg_wheel_speed[:, 1], 5)

['__time', '/raptor_dbw_interface/accelerator_pedal_report/control_type/value', '/raptor_dbw_interface/accelerator_pedal_report/driver_activity', '/raptor_dbw_interface/accelerator_pedal_report/enabled', '/raptor_dbw_interface/accelerator_pedal_report/fault_accel_pedal_system', '/raptor_dbw_interface/accelerator_pedal_report/fault_ch1', '/raptor_dbw_interface/accelerator_pedal_report/fault_ch2', '/raptor_dbw_interface/accelerator_pedal_report/header/stamp/nanosec', '/raptor_dbw_interface/accelerator_pedal_report/header/stamp/sec', '/raptor_dbw_interface/accelerator_pedal_report/ignore_driver', '/raptor_dbw_interface/accelerator_pedal_report/pedal_input', '/raptor_dbw_interface/accelerator_pedal_report/pedal_output', '/raptor_dbw_interface/accelerator_pedal_report/rolling_counter', '/raptor_dbw_interface/accelerator_pedal_report/torque_actual', '/raptor_dbw_interface/brake_2_report/estimated_road_slope', '/raptor_dbw_interface/brake_2_report/front_brake_pressure', '/raptor_dbw_interface

In [4]:
#Wheel speed with residuals
thr = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/accelerator_pedal_report/pedal_output']))
desired_vel = normalize_data(imu_times, np.array(imu_msgs['/vehicle/desired_velocity_readout/data']))
steering = normalize_data(imu_times, np.array(imu_msgs['/joystick/steering_cmd/data']))

f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(15)
f2.set_figheight(10)
idxs = np.where(np.logical_and(rear_avg_wheel_speed[:, 0]>= 1200, rear_avg_wheel_speed[:, 0]<= 1380))
axs[0].scatter(rear_avg_wheel_speed[:, 0][idxs], rear_avg_wheel_speed[:, 1][idxs], s=.5, alpha=0.5, c='orange', label="Rear Avg Velocity")
axs[0].plot(filtered_rear_avg_wheel_speed[:, 0][idxs], filtered_rear_avg_wheel_speed[:, 1][idxs], 'b', label="Filtered Velocity")
axs[0].plot(desired_vel[:, 0][idxs], desired_vel[:, 1][idxs], 'r', label="Desired Velocity")
axs[0].legend()
axs[0].set_ylabel("Speed (MPH)")

axs[1].plot(filtered_rear_avg_wheel_speed[:, 0][idxs], desired_vel[:, 1][idxs] - filtered_rear_avg_wheel_speed[:, 1][idxs], 'b', linewidth = 0.5, label="Residual Velocity")
ax2 = axs[1].twinx()
ax2.plot(steering[:, 0][idxs], steering[:, 1][idxs] / 2, 'r', linewidth = 0.5, label="Steering Angle")
ax2.set_ylabel("Steering Angle (deg)")
ax2.set_ylim(-2, 2)

ax2.legend(loc="upper right")
axs[1].legend(loc="upper left")
axs[1].set_ylabel("Speed (MPH)")
# axs[1].set_xlabel("Time (Seconds)")
axs[1].set_ylim(-10, 10)

# axs[2].set_ylabel("Steering Angle (deg)")
# axs[2].set_xlabel("Time (Seconds)")
# axs[2].set_ylim(-5, 5)
for ax in axs:
    ax.tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True)


# plt.savefig("/mnt/c/Users/johnl/OneDrive/College/!Current Semester/CS Research/lvms/speed_with_steering.pdf", format='pdf', dpi=20000)

# print(len(filtered_rear_avg_wheel_speed[4:-2, 0]))
# print(len(desired_vel[:, 0]))
    
plt.show()

In [5]:
rear_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/rear_brake_pressure']))
front_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/front_brake_pressure']))
# des_brake_pressure = normalize_data(imu_times, np.array(imu_msgs['/joystick/brake_cmd/data']))
des_brake_pressure = normalize_data(imu_times, np.array(imu_msgs['/joystick/brake_cmd/data']))
accel_x = normalize_data(imu_times, np.array(imu_msgs['/accel/filtered/accel/accel/linear/x']))
nov_accel_x = normalize_data(imu_times, np.array(imu_msgs['/novatel_bottom/rawimu/linear_acceleration/y']))
rpm = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/engine_speed']))
# v_accel_x = normalize_data(imu_times, np.array(imu_msgs['/vectornav/raw/attitude/linearaccelbody/x']))
thr_pedal = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/throttle_pedal']))
thr_aim = normalize_data(raptor_times, raptor_msgs["/raptor_dbw_interface/motec_report/throttle_aim_state"])
gear = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/gear']))


gear_ratios = np.array([0, 2.9167, 1.875, 1.3809, 1.1154, 0.96, 0.8889])
final_drive = 3.0
req_brake_torque = (0.0050222 * 0.45 * 0.133 * (gaussian_filter1d(des_brake_pressure[:, 1], 1)) * 4) * -1
engine_brake = np.ones_like(gear[:,1]) * 250_000 * 0.002 / ( 2 * 3.1415926 * 2) * gear_ratios[gear[:, 1].astype(int)] * final_drive
engine_brake[np.where(rpm[:, 1] < 1500)[0]] = 0
engine_brake[np.where(thr_aim[:,1] != 1)[0]] = 0


brake_torque = (0.0050222 * 0.37 * 0.133 * (gaussian_filter1d(front_brake_pressure[:, 1], 1) * 1000) * 2) + (0.0050222 * 0.37 * 0.133 * (gaussian_filter1d(rear_brake_pressure[:, 1], 1) * 1000) * 2 )
rolling_res = np.ones_like(brake_torque) * 9.81 * 0.015
rolling_res[np.where(rear_avg_wheel_speed[:-10, 1] < 0.5)[0]] = 0
brake_torque[np.where(rear_avg_wheel_speed[:-10, 1] < 0.5)[0]] = 0
mass = 750 + ((0.5 * (rear_avg_wheel_speed[:, 1] * 0.44704)**2 * 2.27 * 0.15 * 1.22) / 9.81)
drag = (0.5 * (rear_avg_wheel_speed[:, 1] * 0.44704)**2 * 2.27 * 0.37 * 1.22)
# engine_brake[np.where(rpm[:, 1] < 1600)[0]] = 0

#Deceleration vs Brake Pressure
idxs = np.where(np.logical_and(rear_brake_pressure[:, 0]>= 1390, rear_brake_pressure[:, 0]<= 1430))
idx2 = np.where(np.logical_and(nov_accel_x[:, 0]>= 1390, nov_accel_x[:, 0]<= 1430))
f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(10)
f2.set_figheight(5)
axs[0].plot(rear_brake_pressure[:, 0][idxs], rear_brake_pressure[:, 1][idxs] / 1000, 'b', linewidth=0.5, label="Rear Brake Pressure")
axs[0].plot(front_brake_pressure[:, 0][idxs], front_brake_pressure[:, 1][idxs] / 1000, 'r', linewidth=0.5, label="Front Brake Pressure")
axs[0].plot(des_brake_pressure[:, 0][idxs], des_brake_pressure[:, 1][idxs] / 1000000, 'g', linewidth=0.5, label="Desired Brake Pressure")
# axs[0].plot(rear_avg_wheel_speed[:, 0][idxs], rear_avg_wheel_speed[:, 1][idxs], 'k', linewidth=0.5, label="Speed")
# axs[0].plot(rpm[:, 0][idxs], rpm[:, 1][idxs] / 100, 'orange', linewidth=0.5, label="rpm")

# ax.plot(front_brake_pressure[:, 0], front_brake_pressure[:, 1] / ( rear_brake_pressure[:, 1] + front_brake_pressure[:, 1]) , 'g', label="")
axs[0].legend()
axs[0].set_ylabel("Pressure (Mpa)")
# axs[0].set_xlabel("Time (Seconds)")

axs[1].plot(accel_x[:, 0][idxs], accel_x[:, 1][idxs], 'b', linewidth=0.5, label="Linear Accel, X")
axs[1].plot(nov_accel_x[:, 0][idx2], gaussian_filter1d(-nov_accel_x[:, 1][idx2], 10), 'r', linewidth=0.5, label="Nov Filtered Accel, X")
axs[1].plot(des_brake_pressure[:, 0][idxs], (gaussian_filter1d((- engine_brake[idxs] - brake_torque[idxs]) / (0.3113 * mass[idxs]) - (drag[idxs] / mass[idxs]) - rolling_res[idxs], 2)) , 'k', linewidth=0.5, label="Calculated Accel, X")
# axs[1].plot(des_brake_pressure[:, 0][idxs], (gaussian_filter1d((- engine_brake[idxs] - des_brake_pressure[:, 1][idxs] / 1000) / (0.3113 * mass[idxs]) - (drag[idxs] / mass[idxs]) - rolling_res[idxs], 2)), 'g', linewidth=0.5, label="Calc Req Accel, X")
# axs[1].plot(v_accel_x[:, 0][idxs], uniform_filter1d(v_accel_x[:, 1][idxs],50, mode='constant', origin=(-50//2)), 'r', linewidth=0.5, label="Vectornav Accel, X")
axs[1].legend()
axs[1].set_ylabel("Acceleration (m/s^2)")
axs[1].set_xlabel("Time (Seconds)")

for ax in axs:
    ax.tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True)

# axs[1].set_ylim(15, -15)
# plt.savefig("/mnt/c/Users/johnl/OneDrive/College/!Current Semester/CS Research/lvms/lvms_120_brake_accel_with_more.pdf", format='pdf', dpi=20000)

plt.show()

In [6]:

thr = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/accelerator_pedal_report/pedal_output']))
gear = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/gear']))

#Deceleration vs Brake Pressure
idxs = np.where(np.logical_and(rear_brake_pressure[:, 0]>= 960, rear_brake_pressure[:, 0]<= 980))

f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(10)
f2.set_figheight(5)
MPH2MS = 0.44704
axs[0].scatter(thr[:, 0][idxs], thr[:, 1][idxs], s=.5, alpha=.5, c='orange', label="Unfiltered Throttle")
axs[0].plot(filtered_rear_avg_wheel_speed[:, 0][idxs], filtered_rear_avg_wheel_speed[:, 1][idxs] * MPH2MS, 'orange', label="Velocity")
axs[0].plot(thr[:, 0][idxs], gaussian_filter1d(thr[:, 1][idxs], 5), 'b', label="Throttle")
axs[0].set_ylim(0, 100)
axs[0].legend()
axs[0].set_ylabel("% and m/s")
# axs[0].set_xlabel("Time (Seconds)")

axs[1].plot(gear[:, 0][idxs], gear[:, 1][idxs], 'r', label="Gear")
axs[1].legend()
axs[1].set_ylabel("Gear")
axs[1].set_xlabel("Time (Seconds)")
axs[1].set_ylim(0, 6)

for ax in axs:
    ax.tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True)

# plt.savefig("/mnt/c/Users/johnl/OneDrive/College/!Current Semester/CS Research/lvms_brake_throttle_percent.pdf", format='pdf', dpi=20000)


plt.show()

In [7]:
engine_torque = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/torque_wheels']))
# front_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/front_brake_pressure']))
rear_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/rear_brake_pressure']))
front_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/front_brake_pressure']))
# des_brake_pressure = normalize_data(imu_times, np.array(imu_msgs['/long_control/brake_cmd/data']))
accel_x = normalize_data(imu_times, np.array(imu_msgs['/accel/filtered/accel/accel/linear/x']))
# v_accel_x = normalize_data(imu_times, np.array(imu_msgs['/vectornav/raw/attitude/linearaccelbody/x']))
gear = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/gear']))
e_eff = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/engine_efficiency']))
nov = normalize_data(imu_times, np.array(imu_msgs["/novatel_bottom/rawimu/linear_acceleration/y"]))
boost = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/boost_pressure']))
boost_target = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/boost_aim']))
map = normalize_data(raptor_times, raptor_msgs["/raptor_dbw_interface/motec_report/inlet_manifold_pressure"])
rpm = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/engine_speed']))
throttle = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/throttle_position']))
thr_pedal = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/throttle_pedal']))
thr_aim = normalize_data(raptor_times, raptor_msgs["/raptor_dbw_interface/motec_report/throttle_aim_state"])

gear_ratios = np.array([0, 2.9167, 1.875, 1.3809, 1.1154, 0.96, 0.8889])
final_drive = 3.0
engine_torque[:, 1] = engine_torque[:, 1] * gear_ratios[gear[:, 1].astype(int)] * final_drive

torque_matrix = np.loadtxt('dyno_matrix_maker/new_data/output_matrix.csv', delimiter=',')
torques = np.array(torque_matrix[rpm[:, 1].astype(int) - 1600, throttle[:, 1].astype(int)])
torques *= final_drive * gear_ratios[gear[:, 1].astype(int)] * 0.8
torques[rpm[:, 1] < 1600] = 0.0
torques[np.where(thr_pedal[:, 1] < 5)[0]] = 0.0

c1 = .01

engine_brake = np.ones_like(engine_torque[:,1]) * c1 * rpm[:, 1] * gear_ratios[gear[:, 1].astype(int)] * final_drive
engine_brake[np.where(rpm[:, 1] < 1500)[0]] = 0
engine_brake[np.where(thr_aim[:, 1] != 1)[0]] = 0


brake_torque = (0.0050222 * 0.45 * 0.133 * (gaussian_filter1d(front_brake_pressure[:, 1], 1) * 1000) * 2) + (0.0050222 * 0.45 * 0.133 * (gaussian_filter1d(rear_brake_pressure[:, 1], 1) * 1000) * 2 )
rolling_res = np.ones_like(brake_torque) * 9.81 * 0.015
rolling_res[np.where(rear_avg_wheel_speed[:-10, 1] < 0.5)[0]] = 0
brake_torque[np.where(rear_avg_wheel_speed[:-10, 1] < 0.5)[0]] = 0
mass = 750 + ((0.5 * (rear_avg_wheel_speed[:, 1] * 0.44704)**2 * 2.27 * 0.15 * 1.22) / 9.81)
drag = (0.5 * (rear_avg_wheel_speed[:, 1] * 0.44704)**2 * 2.27 * 0.24 * 1.22)


#Deceleration vs Brake Pressure
start = 1000
# stop = engine_torque[-1, 0] - 10
stop = 1350
idxs = np.where(np.logical_and(engine_torque[:, 0]>= start, engine_torque[:, 0]<= stop))
idx3 = np.where(np.logical_and(nov[:, 0]>= start, nov[:, 0]<= stop))
f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(10)
f2.set_figheight(5)

# axs[0].plot(calc_torque[:, 0][idx2], calc_torque[:, 1][idx2], 'r', linewidth=0.5, label="Calculated Torque")
axs[0].plot(accel_x[:, 0][idxs], accel_x[:, 1][idxs], 'b', linewidth=0.5, label="EKF Accel")
# axs[0].plot(v_accel_x[:, 0][idx2], gaussian_filter1d(v_accel_x[:, 1][idx2],20), 'g', linewidth=0.5, label="Vec Accel")
axs[0].plot(nov[:, 0][idx3], gaussian_filter1d(nov[:, 1][idx3] * -1, 10), 'k', linewidth=0.5, label="Novatel Accel")
axs[0].plot(engine_torque[:, 0][idxs], gaussian_filter1d((engine_torque[:, 1][idxs] - brake_torque[idxs]) / (0.3113 * mass[idxs])
                                                          - drag[idxs]/mass[idxs] - rolling_res[idxs], 2), 'g', linewidth=0.5, label="Measured Calc Accel")
axs[0].plot(engine_torque[:, 0][idxs], gaussian_filter1d((torques[idxs] - engine_brake[idxs]  - brake_torque[idxs]) / (0.3113 * mass[idxs])
                                                          - drag[idxs]/mass[idxs] - rolling_res[idxs], 2), 'r', linewidth=0.5, label="Calculated Accel")
# axs[0].plot(calc_torque[:, 0][idxs], shifted_toruqe[idxs], 'g', linewidth=0.5, label="Shifted Torque")
axs[0].legend()
axs[0].set_ylabel("Acceleration ms^2")
# axs[0].set_xlabel("Time (Seconds)")
# print(e_eff[:, 1])
axs[1].plot(throttle[:, 0][idxs], throttle[:, 1][idxs], 'b', linewidth=0.5, label="Throttle %")
# axs[1].plot(map[:, 0][idxs], map[:, 1][idxs], 'g', linewidth=0.5, label="Map %")
# axs[1].plot(nov[:, 0][idx3], engine_torque[:, 1][idxs] - calc_torque[:, 1][idxs], 'b', linewidth=0.5, label="Originial")
# axs[1].plot(boost_target[:, 0][idxs], boost_target[:, 1][idxs], 'b', linewidth=0.5, label="target")
# axs[1].plot(boost[:, 0][idxs], boost[:, 1][idxs], 'r', linewidth=0.5, label="boost")

# axs[1].plot(calc_torque[:, 0][idxs], engine_torque[:, 1][idxs] - shifted_toruqe[idxs], 'g', linewidth=0.5, label="Shifted")
# ax2 = axs[1].twinx()
# ax2.plot(rpm[:, 0][idxs], rpm[:, 1][idxs], 'r', linewidth=0.5, label="RPM")
# ax2.set_ylabel("RPM")
# ax2.legend()

ax3 = axs[1].twinx()
ax3.plot(rear_avg_wheel_speed[:, 0][idxs], rear_avg_wheel_speed[:, 1][idxs], 'k', linewidth=0.5, label="MPH")
ax3.set_ylabel("MPH")
ax3.legend()
# axs[1].plot(v_accel_x[:, 0][idxs], gaussian_filter1d(v_accel_x[:, 1][idxs],10), 'r', linewidth=0.5, label="Vectornav Accel, X")
axs[1].legend()
axs[1].set_ylim(0, 100)
axs[1].set_ylabel("Residuals")
axs[1].set_xlabel("Time (Seconds)")

for ax in axs:
    ax.tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True)

# axs[1].set_ylim(15, -15)
# plt.savefig("/mnt/c/Users/johnl/OneDrive/College/!Current Semester/CS Research/lvms/torque_at_120_before_brake_good_at_high_speeds.pdf", format='pdf', dpi=20000)

plt.show()

In [55]:
engine_torque = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/torque_wheels']))
# front_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/front_brake_pressure']))
rear_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/rear_brake_pressure']))
front_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/front_brake_pressure']))
des_brake_pressure = normalize_data(imu_times, np.array(imu_msgs['/joystick/brake_cmd/data']))
accel_x = normalize_data(imu_times, np.array(imu_msgs['/accel/filtered/accel/accel/linear/x']))
# v_accel_x = normalize_data(imu_times, np.array(imu_msgs['/vectornav/raw/attitude/linearaccelbody/x']))
gear = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/gear']))
e_eff = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/engine_efficiency']))
nov = normalize_data(imu_times, np.array(imu_msgs["/novatel_bottom/rawimu/linear_acceleration/y"]))
boost = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/boost_pressure']))
boost_target = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/boost_aim']))
map = normalize_data(raptor_times, raptor_msgs["/raptor_dbw_interface/motec_report/inlet_manifold_pressure"])
rpm = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/engine_speed']))
throttle = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/throttle_position']))
thr_pedal = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/throttle_pedal']))
thr_aim = normalize_data(raptor_times, raptor_msgs["/raptor_dbw_interface/motec_report/throttle_aim_state"])

gear_ratios = np.array([0, 2.9167, 1.875, 1.3809, 1.1154, 0.96, 0.8889])
final_drive = 3.0
engine_torque[:, 1] = engine_torque[:, 1] * gear_ratios[gear[:, 1].astype(int)] * final_drive

torque_matrix = np.loadtxt('dyno_matrix_maker/new_data/output_matrix.csv', delimiter=',')
torques = np.array(torque_matrix[rpm[:, 1].astype(int) - 1600, throttle[:, 1].astype(int)])
torques *= final_drive * gear_ratios[gear[:, 1].astype(int)] * 0.8
torques[rpm[:, 1] < 1600] = 0.0
torques[np.where(thr_pedal[:, 1] < 5)[0]] = 0.0

c1 = .01

engine_brake = np.ones_like(engine_torque[:,1]) * c1 * rpm[:, 1] * gear_ratios[gear[:, 1].astype(int)] * final_drive
engine_brake[np.where(rpm[:, 1] < 1500)[0]] = 0
engine_brake[np.where(thr_aim[:, 1] != 1)[0]] = 0


brake_torque = (0.0050222 * 0.45 * 0.133 * (gaussian_filter1d(front_brake_pressure[:, 1], 1) * 1000) * 2) + (0.0050222 * 0.45 * 0.133 * (gaussian_filter1d(rear_brake_pressure[:, 1], 1) * 1000) * 2 )
rolling_res = np.ones_like(brake_torque) * 9.81 * 0.015
rolling_res[np.where(rear_avg_wheel_speed[:-10, 1] < 0.5)[0]] = 0
brake_torque[np.where(rear_avg_wheel_speed[:-10, 1] < 0.5)[0]] = 0
mass = 750 + ((0.5 * (rear_avg_wheel_speed[:, 1] * 0.44704)**2 * 2.27 * 0.15 * 1.22) / 9.81)
drag = (0.5 * (rear_avg_wheel_speed[:, 1] * 0.44704)**2 * 2.27 * 0.24 * 1.22)

total_gear_ratio = final_drive * gear_ratios[gear[:, 1].astype(int)]
wheel_rad = 0.3113
b1 = 1.9
b2 = b1
a1 = 0.08
a2 = 1.2
c1 = 3.1
c2 = 0.01

start = 200
# stop = engine_torque[-1, 0] - 10
stop = 1480

idxs = np.where(np.logical_and(engine_torque[:, 0]>= start, engine_torque[:, 0]<= stop))
idx3 = np.where(np.logical_and(nov[:, 0]>= start, nov[:, 0]<= stop))
engine_decel = (c2 * total_gear_ratio[idxs] * rpm[:, 1][idxs])
engine_decel[thr_pedal[:, 1][idxs] > 3] = 0
forces = (c1 * torques[idxs]) - engine_decel - (b1 * front_brake_pressure[:, 1][idxs] + b2 * rear_brake_pressure[:, 1][idxs]) - (rear_avg_wheel_speed[:, 1][idxs]**2 * a1) - (rear_avg_wheel_speed[:, 1][idxs] * a2)

f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(10)
f2.set_figheight(5)

# axs[0].plot(calc_torque[:, 0][idx2], calc_torque[:, 1][idx2], 'r', linewidth=0.5, label="Calculated Torque")
axs[0].plot(accel_x[:, 0][idxs], accel_x[:, 1][idxs], 'b', linewidth=0.5, label="EKF Accel")
# axs[0].plot(v_accel_x[:, 0][idx2], gaussian_filter1d(v_accel_x[:, 1][idx2],20), 'g', linewidth=0.5, label="Vec Accel")
axs[0].plot(nov[:, 0][idx3], gaussian_filter1d(nov[:, 1][idx3] * -1, 10), 'k', linewidth=0.5, label="Novatel Accel")
axs[0].plot(engine_torque[:, 0][idxs], gaussian_filter1d((engine_torque[:, 1][idxs] - brake_torque[idxs]) / (0.3113 * mass[idxs])
                                                          - drag[idxs]/mass[idxs] - rolling_res[idxs], 2), 'g', linewidth=0.5, label="Measured Calc Accel")
axs[0].plot(engine_torque[:, 0][idxs], forces/750, 'r', linewidth=0.5, label="Calculated Accel")
# axs[0].plot(calc_torque[:, 0][idxs], shifted_toruqe[idxs], 'g', linewidth=0.5, label="Shifted Torque")
axs[0].legend()
axs[0].set_ylabel("Acceleration ms^2")
# axs[0].set_xlabel("Time (Seconds)")
# print(e_eff[:, 1])
axs[1].plot(throttle[:, 0][idxs], thr_pedal[:, 1][idxs], 'b', linewidth=0.5, label="Throttle %")
# axs[1].plot(map[:, 0][idxs], map[:, 1][idxs], 'g', linewidth=0.5, label="Map %")
# axs[1].plot(nov[:, 0][idx3], engine_torque[:, 1][idxs] - calc_torque[:, 1][idxs], 'b', linewidth=0.5, label="Originial")
# axs[1].plot(boost_target[:, 0][idxs], boost_target[:, 1][idxs], 'b', linewidth=0.5, label="target")
# axs[1].plot(boost[:, 0][idxs], boost[:, 1][idxs], 'r', linewidth=0.5, label="boost")

# axs[1].plot(calc_torque[:, 0][idxs], engine_torque[:, 1][idxs] - shifted_toruqe[idxs], 'g', linewidth=0.5, label="Shifted")
# ax2 = axs[1].twinx()
# ax2.plot(rpm[:, 0][idxs], rpm[:, 1][idxs], 'r', linewidth=0.5, label="RPM")
# ax2.set_ylabel("RPM")
# ax2.legend()

ax3 = axs[1].twinx()
ax3.plot(rear_avg_wheel_speed[:, 0][idxs], rear_avg_wheel_speed[:, 1][idxs], 'k', linewidth=0.5, label="MPH")
ax3.set_ylabel("MPH")
ax3.legend()
# axs[1].plot(v_accel_x[:, 0][idxs], gaussian_filter1d(v_accel_x[:, 1][idxs],10), 'r', linewidth=0.5, label="Vectornav Accel, X")
axs[1].legend()
axs[1].set_ylim(0, 100)
axs[1].set_ylabel("Residuals")
axs[1].set_xlabel("Time (Seconds)")

for ax in axs:
    ax.tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True)

# axs[1].set_ylim(15, -15)
# plt.savefig("/mnt/c/Users/johnl/OneDrive/College/!Current Semester/CS Research/lvms/torque_at_120_before_brake_good_at_high_speeds.pdf", format='pdf', dpi=20000)

plt.show()